In [7]:
import os, sys, email, datetime, pprint, re, time, html, pickle
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [8]:
### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")
from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

In [9]:
from db import connect
engine = connect()
mail_df = pd.read_sql(sql='SELECT DISTINCT id, body FROM mail',con=engine, index_col=None)

In [10]:
start = time.time()
# MinIEにかける
import requests
import json

pattern = re.compile('\W+')
count = 0
results = {}

for value in mail_df.values:
    uid = value[0]
    body = value[1]
    doc = nlp(body)
    result_list = []
    not_sent_count = 0

    for sent in doc.sents:
        sent_text = sent.text
        if sent_text.isnumeric() or pattern.fullmatch(sent_text) or len(sent_text.split()) <= 5:
            not_sent_count = not_sent_count + 1
            pass
        else:
            match = re.search(re.escape(sent_text), doc.text)
            offset_info = {'sentence':sent_text, 'start':match.start(), 'end':match.end()}
            try:
                response = requests.post('http://localhost:8080/minie/query', data=sent.text.encode('utf-8'))
                result = response.json()
                if result['facts'] != []:
                    offset_info.update(result)
                    result_list.append(offset_info)
            except json.JSONDecodeError:
                pass
    
    results[uid] = result_list
    count = count + 1
    print('{} mails finished, {} sentences were excluded.'.format(count,not_sent_count))

total = time.time() - start
print(total/60)

1 mails finished, 3 sentences were excluded.
2 mails finished, 1 sentences were excluded.
3 mails finished, 12 sentences were excluded.
4 mails finished, 2 sentences were excluded.
5 mails finished, 23 sentences were excluded.
6 mails finished, 19 sentences were excluded.
7 mails finished, 13 sentences were excluded.
8 mails finished, 3 sentences were excluded.
9 mails finished, 4 sentences were excluded.
10 mails finished, 42 sentences were excluded.
11 mails finished, 3 sentences were excluded.
12 mails finished, 40 sentences were excluded.
13 mails finished, 4 sentences were excluded.
14 mails finished, 0 sentences were excluded.
15 mails finished, 1 sentences were excluded.
16 mails finished, 11 sentences were excluded.
17 mails finished, 99 sentences were excluded.
18 mails finished, 52 sentences were excluded.
19 mails finished, 31 sentences were excluded.
20 mails finished, 11 sentences were excluded.
21 mails finished, 1 sentences were excluded.
22 mails finished, 30 sentences 

In [11]:
len(results)

370

In [12]:
# 保存
with open('/Users/taroaso/myprojects/OpenIE/trec/output/openie_result.pickle', mode='wb') as f:
    pickle.dump(results,f)